# Reduce catalog based on SEDs
## January 2024
### Bethany Ludwig

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn as sns
import numpy as np 
import os, glob
import matplotlib
from astropy.io import ascii
from matplotlib import rc
from scipy import stats
from astropy.coordinates import SkyCoord
np.seterr(invalid='ignore')

# Set dir depending on machine 
directory = '/home/bethany/Projects/0_Data/0_SUMS_Catalogs/CandidateCatalog/'
# directory = '/Volumes/Untitled/0_Data/SUMS_Catalogs/Candidates/'
synth_photom_file = '/home/bethany/Projects/0_Data/1_Models/Stripped_Stars/photometry_CMFGEN_composites.txt'
# synth_photom_file = '/Volumes/Untitled/0_Data/Models/photometry_CMFGEN_composites.txt'

## Load in photometry

In [2]:
# Read in magnitude reduced files 
df = pd.read_csv(directory+'1_magnitude_reduced.csv') 
print(df.galaxy.value_counts())

# Make a key
df['key'] = np.arange(df.shape[0])

# Rename filters for simplicity, we can rename them back at the end
df = df.rename(columns={'uvw2_dered':'w2','uvm2_dered':'m2','uvw1_dered':'w1',
                                          'U_dered':'u','B_dered':'b','V_dered':'v','I_dered':'i',
                                          'uvw2 - b':'w2 - b', 'uvw2 - v':'w2 - v', 'uvw2 - i':'w2 - i',
                                          'uvw1 - b':'w1 - b', 'uvw1 - v':'w1 - v', 'uvw1 - i':'w1 - i', 
                                          'uvm2 - b':'m2 - b', 'uvm2 - v':'m2 - v', 'uvm2 - i':'m2 - i'})

# Select keys
optical_columns = ['u','b','v','i']
uv_columns = ['w2','m2','w1']
color_columns = ['w2 - b', 'w2 - v', 'w2 - i',
                'm2 - b', 'm2 - v', 'm2 - i',
                'w1 - b', 'w1 - v', 'w1 - i']

# Make sure we have all the discovery stars except Star 15 (and now Star 19) which we lost in the last notebook 
def check_stars(df):
    # Make sure we have all the discovery stars 
    discovery_names = ['Star_1','Star_2','Star_3','Star_4','Star_5','Star_6','Star_7','Star_8','Star_9','Star_10','Star_11','Star_12','Star_13','Star_14','Star_16','Star_17','Star_18','Star_20','Star_21','Star_22','Star_23','Star_24','Star_25']
    c = 0
    for star in discovery_names:
        if star not in df.discovery_name.unique():
            print (star)
            c += 1
    if c == 0:
        print ("All Remaining Discovery Stars After Magnitude Reductions Have Been Found")

# Print how many stars we have
print("How many stars we have: ",df.shape[0])

# Read in synthetic photometry
names = ['Minit_strip','M_MS','frac_MS','U','B','V','R','I','UVM2','UVW2','UVW1']

sf = pd.read_csv(synth_photom_file,comment='#',delimiter='\t',names=names)

# Make dictionary of the minimum and maximum difference between different filters
max_mag_err = 0.217
min_diff = {'w2 - m2': np.min(sf.UVW2-sf.UVM2) - max_mag_err,
            'm2 - w1': np.min(sf.UVM2-sf.UVW1) - max_mag_err,
            'w1 - u': np.min(sf.UVW1-sf.U) - max_mag_err,
            'u - b': np.min(sf.U-sf.B) - max_mag_err,
            'b - v': np.min(sf.B-sf.V) - max_mag_err,
            'v - i': np.min(sf.V-sf.I) - max_mag_err,
            # Additional UV - Optical Filters
            'w1 - b': np.min(sf.UVW1-sf.B) - max_mag_err,
            'm2 - u': np.min(sf.UVM2-sf.U) - max_mag_err,
            'm2 - b': np.min(sf.UVM2-sf.B) - max_mag_err,
           }

max_diff = {'w2 - m2': np.max(sf.UVW2-sf.UVM2) + max_mag_err,
            'm2 - w1': np.max(sf.UVM2-sf.UVW1) + max_mag_err,
            'w1 - u': np.max(sf.UVW1-sf.U) + max_mag_err,
            'u - b': np.max(sf.U-sf.B) + max_mag_err,
            'b - v': np.max(sf.B-sf.V) + max_mag_err,
            'v - i': np.max(sf.V-sf.I) + max_mag_err,
            # Additional UV - Optical Filters
            'w1 - b': np.max(sf.UVW1-sf.B) + max_mag_err,
            'm2 - u': np.max(sf.UVM2-sf.U) + max_mag_err,
            'm2 - b': np.max(sf.UVM2-sf.B) + max_mag_err,
           }

check_stars(df)

galaxy
lmc    8906
smc    4447
Name: count, dtype: int64
How many stars we have:  13353
All Remaining Discovery Stars After Magnitude Reductions Have Been Found


#### If blue due to one filter, check that that filter is not super different from adjacent filters

In [3]:
size = df.shape[0]
# Get rows where only one filter made it blue
blue_in_one = df[df['n_blue'] <= 3].copy()

# Compare adjacent mags to see if the filter is super different from the sed 
bad_filter_index = []

# For the filters on the ends, uvw2 and i, you only need to compare one color
filter_1 = ['w2','v']
filter_2 = ['m2','i']
filter_color = ['w2','i']

for f1, f2, fc in zip(filter_1,filter_2,filter_color):
    # Get all color combos for a specific filter 
    # For example, get all the w2 - b, w2 - v, w2 - i for w2
    c = list(filter(lambda x: fc in x,color_columns))
    # Get everything where the respective filter is blue 
    blue = blue_in_one[(blue_in_one[c[0]] == 'blue') & (blue_in_one[c[1]] == 'blue') & (blue_in_one[c[2]] == 'blue')]
    # Calculate the difference between the filter and its adjacent filter
    diff = blue[f'{f1}'] - blue[f'{f2}']
    # Compare to synthetic photometry      
    bad = blue[(diff > max_diff[f'{f1} - {f2}']) | (diff < min_diff[f'{f1} - {f2}']) ]
    bad_keys = bad['key'].values
    print('')
    bad_filter_index.append(bad_keys)

# For all the other filters you need to compare whats on the left and the right 
filters = ['w2','m2','w1','u','b','v','i']          

for i in np.arange(6):
    # Skip the ends
    if filters[i] == 'w2' or filters[i] == 'i' or filters[i] == 'u':
        continue
    # The filter you're considering
    f0 = filters[i]
    # The previous filter
    f1 = filters[i-1]
    # The next filter
    f2 = filters[i+1]
    print(f'Solving {f0}')
    # Get colors 
    c = list(filter(lambda x: f0 in x,color_columns))
    print(f'Checking Colors: {c}')
    # Get everything where the respective filter is blue 
    blue = blue_in_one[(blue_in_one[c[0]] == 'blue') & (blue_in_one[c[1]] == 'blue') & (blue_in_one[c[2]] == 'blue')] 
    # Calculate differences
    print(f'Calculating difference for: "{f1} - {f0}"')
    diff_1 = blue[f'{f1}'] - blue[f'{f0}']
    print(f'Calculating difference for: "{f0} - {f2}"')
    diff_2 = blue[f'{f0}'] - blue[f'{f2}']
    # Calculate synthetic differences
    synth_1 = [min_diff[f'{f1} - {f0}'], max_diff[f'{f1} - {f0}']]
    synth_2 = [min_diff[f'{f0} - {f2}'], max_diff[f'{f0} - {f2}']]
    # Compare                    The Left                                        The Right
    bad = blue[(diff_1 < synth_1[0]) | (diff_1 > synth_1[1]) | (diff_2 < synth_2[0]) | (diff_2 > synth_2[1])]
    bad_keys = bad['key'].values
    if len(bad_keys) == 0: 
        print('Check Filter')
        print(f0)
    print('')
    # Append
    bad_filter_index.append(bad_keys)
    
# Flatten the index
bad_filter_index = [item for row in bad_filter_index for item in row]
# Sort the index 
bad_filter_index = np.sort(bad_filter_index)

df = df[~df['key'].isin(bad_filter_index)]

print(f'How many we have: ',df.shape[0])
print(f'How many we lost: ',size-df.shape[0])
check_stars(df)



Solving m2
Checking Colors: ['m2 - b', 'm2 - v', 'm2 - i']
Calculating difference for: "w2 - m2"
Calculating difference for: "m2 - w1"

Solving w1
Checking Colors: ['w1 - b', 'w1 - v', 'w1 - i']
Calculating difference for: "m2 - w1"
Calculating difference for: "w1 - u"

Solving b
Checking Colors: ['w2 - b', 'm2 - b', 'w1 - b']
Calculating difference for: "u - b"
Calculating difference for: "b - v"

Solving v
Checking Colors: ['w2 - v', 'm2 - v', 'w1 - v']
Calculating difference for: "b - v"
Calculating difference for: "v - i"

How many we have:  11129
How many we lost:  2224
All Remaining Discovery Stars After Magnitude Reductions Have Been Found


#### Remove sources that have a uv-optical mismatch 
There's a sharp enough discontinuity between adjacent(ish) uv and optical filters that we think it is not the same source. Using synthetic photometry Ylva created we determine what the max mag diff between UV and Optical filters should be and restrict based on that.

In [4]:
size = df.shape[0]
# Mismatch #1: UV and Optical components of SED are fairly flat but there's a large discrepency between them

# If all the mags are within the max allowed error, we consider it 'flat'
flat_std = max_mag_err
flat_mag_jump = 1. + max_mag_err

flat_jump_index = []

for ind,row in df.iterrows():
    uv = row[uv_columns].values
    opt = row[optical_columns].values
    
    if np.nanstd(uv) < flat_std and np.nanstd(opt) < flat_std:
        # If you have a flat sed look for a big jump 
        if np.abs(np.nanmean(uv) - np.nanmean(opt)) > flat_mag_jump:
            flat_jump_index.append(row['key'])
            
# Remove from sample 
df = df[~df['key'].isin(flat_jump_index)]

print(f'How many we have: ',df.shape[0])
print(f'How many we lost: ',size - df.shape[0])
check_stars(df)    

How many we have:  10829
How many we lost:  300
All Remaining Discovery Stars After Magnitude Reductions Have Been Found


In [5]:
size = df.shape[0]
# Mismatch #2: Check either UVW1 or UVW2 against U or B to see if there's a large jump 
def not_in_bin(row,uv_filter,optical_filter):
    diff = row[uv_filter] - row[optical_filter]
    if diff < min_diff[f'{uv_filter} - {optical_filter}'] or diff > max_diff[f'{uv_filter} - {optical_filter}']:
        return True
    return False

def exists(row,Filter):
    return np.isfinite(row[Filter])

mismatch_index = []

for i,r in df.iterrows():

    # Ideal case: UVW1, U, and B, all exist. 
    if exists(r,'w1') and exists(r,'u') and exists(r,'b'):
        # If UVW1 - U and UVW1 - B are both not in synthetic bin then drop
        if not_in_bin(r,'w1','u') and not_in_bin(r,'w1','b'):
            mismatch_index.append(r['key'])
    
    # UVW1 and U exist, but B does not.
    if exists(r,'w1') and exists(r,'u') and not exists(r,'b'):
        # If UVW1 - U is not in synthetic bin then drop
        if not_in_bin(r,'w1','u'):
            mismatch_index.append(r['key'])
            
    # UVW1 and B exist, but U does not. 
    if exists(r,'w1') and exists(r,'b') and not exists(r,'u'):
        # If UVW1 - B is not in synthetic bin then drop
        if not_in_bin(r,'w1','b'):
            mismatch_index.append(r['key'])    
    
    # UVM2, U, and B, exist, but UVW1 does not. 
    if exists(r,'m2') and exists(r,'u') and exists(r,'b') and not exists(r,'w1'):
        # If UVM2 - U and UVM2 - B are both not in synthetic bin then drop
        if not_in_bin(r,'m2','u') and not_in_bin(r,'m2','b'):
            mismatch_index.append(r['key']) 
            
    # UVM2 and U exist, but UVW1 and B do not.
    if exists(r,'m2') and exists(r,'u') and not exists(r,'w1') and not exists(r,'b'):
        # If UVM2 - U is not in synthetic bin then drop
        if not_in_bin(r,'m2','u'):
            mismatch_index.append(r['key'])     

    # UVM2 and B exist, but UVW1 and U do not. 
    if exists(r,'m2') and exists(r,'b') and not exists(r,'w1') and not exists(r,'u'):
        # If UVM2 - B is not in synthetic bin then drop
        if not_in_bin(r,'m2','b'):
            mismatch_index.append(r['key'])    
            
df = df[~df['key'].isin(mismatch_index)]

print(f'How many we have: ',df.shape[0])
print(f'How many were categorized as mismatched: ',size - df.shape[0])
check_stars(df)  

How many we have:  9645
How many were categorized as mismatched:  1184
All Remaining Discovery Stars After Magnitude Reductions Have Been Found


#### Remove sources that weren't classified as a mismatch but do increase in red filters.
What are these guys?

In [6]:
size = df.shape[0]
red_index = []

for ind, row in df.iterrows():
    y = np.array(row[optical_columns].values).astype(float)
    # Calculate derivatives
    d = np.diff(y)
    # If derivatives are negative 2 or more times (indicating positive slope for mags) then save
    n = len(d[np.isfinite(d) & (d<0)])
    if n >= 2:
        red_index.append(row['key'])

df = df[~df['key'].isin(red_index)].reset_index(drop=True)

print(f'How many we have: ',df.shape[0])
print(f'How many we lost: ',size-df.shape[0])
check_stars(df)  

How many we have:  6686
How many we lost:  2959
All Remaining Discovery Stars After Magnitude Reductions Have Been Found


#### Remove sources with bumpy SEDs
Using synthetic photometry Ylva created we determine what the max mag diff between adjacent filters should be and restrict based on that.

In [7]:
vf = df.copy()

def not_in_bin(row,uv_filter,optical_filter):
    diff = row[uv_filter] - row[optical_filter]
    if diff < min_diff[f'{uv_filter} - {optical_filter}'] or diff > max_diff[f'{uv_filter} - {optical_filter}']:
        return True
    return False

variable_index = []
filters = ['w2','m2','w1','u','b','v','i']
for ind,row in vf.iterrows():
    # Get the sed
    sed = row[filters]
    # Take the derivatives         
    div = np.abs(np.diff(sed))
    # Compare to allowed differences 
    compare = [not_in_bin(row,f1,f2) for f1,f2 in zip(filters[:-1],filters[1:])]
    # Where do the bumps occur? 
    loc = np.where(compare)[0]
    # How many bumps are there?
    n_bumps = len(loc)
    if n_bumps == 0: 
        continue
    # If there are three or more bumps it's automatically variable 
    if n_bumps >= 3:
        variable_index.append(row.key)
        
    # The location of the bumps tell us which filters are involved 
    # Zero out colors related to that filter and see if it is still blue 
    # For instance: W2 - M2 is the first element in the div array
        
    # If there are two bumps and they are next to each other: 
    if n_bumps == 2 and (loc[1] == loc[0] - 1 or loc[1] == loc[0] + 1): 
        # UVM2 is connected
        if loc[0] == 0 and loc[1] == 1:
            vf.loc[vf.key==row.key,color_columns[1]] = np.nan
        # UVW1 is connected
        if loc[0] == 1 and loc[1] == 2:
            vf.loc[vf.key==row.key,color_columns[2]] = np.nan
        # B is connected, zero out related colors
        if loc[0] == 3 and loc[1] == 4:
            vf.loc[vf.key==row.key,color_columns[4]] = np.nan
        # V is connected, zero out related colors
        if loc[0] == 4 and loc[1] == 5:
            vf.loc[vf.key==row.key,color_columns[5]] = np.nan
    # If there is one bump or they are not connected
    else:
        # UVW2 and UVM2 
        if 0 in loc:
            vf.loc[vf.key==row.key,color_columns[0] + color_columns[1]] = np.nan
        # UVM2 and UVW1 
        if 1 in loc: 
            vf.loc[vf.key==row.key,color_columns[1] + color_columns[2]] = np.nan
        # UVW1 and U (we don't color by U)
        if 2 in loc: 
            vf.loc[vf.key==row.key,color_columns[2]] = np.nan
        # U and B (we don't color by U)
        if 3 in loc: 
            vf.loc[vf.key==row.key,color_columns[4]] = np.nan
        # B and V 
        if 4 in loc: 
            vf.loc[vf.key==row.key,color_columns[4] + color_columns[5]] = np.nan
        # V and I 
        if 5 in loc: 
            vf.loc[vf.key==row.key,color_columns[5] + color_columns[6]] = np.nan
            
# Recalculate to see what is still blue 
n_blue = vf[color_columns].isin(['blue']).sum(axis=1)

# Find out which rows are no longer blue 
nolongerblue = vf.loc[n_blue == 0,'key']

# Combine Variable Index and No Longer Blue, Remove Duplicates
variable_index = np.sort(np.unique(np.append(variable_index,nolongerblue)))

# Drop variables
df = df[~df['key'].isin(variable_index)]
print(f'How many we have: ',df.shape[0])
print(f'How many we lost: ',len(variable_index))
check_stars(df)  

How many we have:  6217
How many we lost:  469
All Remaining Discovery Stars After Magnitude Reductions Have Been Found


#### For what's left, remove 'I' and calculate to see if it still blue

In [8]:
# Change all color combos with 'i' to not be blue. 
i_df = df.copy()

for color in ['w2 - i','w1 - i', 'm2 - i']:
    i_df[color] = i_df[color].replace('blue',np.nan)

# Recalculate to see what is still blue 
n_blue = i_df[color_columns].isin(['blue']).sum(axis=1)

# Find out which rows are no longer blue 
bad_i_index = i_df.loc[n_blue == 0,'key']

# Drop variables
df = df[~df['key'].isin(bad_i_index)]
print(f'How many we have: ',df.shape[0])
print(f'How many we lost: ',len(bad_i_index))
check_stars(df)  

How many we have:  4649
How many we lost:  1568
Star_17
Star_25


#### Recalculate n_bumps for everything in mag_reduced file so that we can save it to use for quality cuts later.

In [9]:
#Reread in and calculate the number of jumps for everything: 
df = pd.read_csv(directory+'1_magnitude_reduced.csv') 

# Rename filters for simplicity, we can rename them back at the end
df = df.rename(columns={'uvw2_dered':'w2','uvm2_dered':'m2','uvw1_dered':'w1',
                                          'U_dered':'u','B_dered':'b','V_dered':'v','I_dered':'i',
                                          'uvw2 - b':'w2 - b', 'uvw2 - v':'w2 - v', 'uvw2 - i':'w2 - i',
                                          'uvw1 - b':'w1 - b', 'uvw1 - v':'w1 - v', 'uvw1 - i':'w1 - i', 
                                          'uvm2 - b':'m2 - b', 'uvm2 - v':'m2 - v', 'uvm2 - i':'m2 - i'})

#Recalculate the number of bumps for this sample and save it as a column: 
filters = ['w2','m2','w1','u','b','v','i']
n_bumps = []
for ind,row in df.iterrows():
    # Get the sed
    sed = row[filters]
    # Take the derivatives         
    div = np.abs(np.diff(sed))
    # Compare to allowed differences 
    compare = [not_in_bin(row,f1,f2) for f1,f2 in zip(filters[:-1],filters[1:])]
    # Where do the bumps occur? 
    loc = np.where(compare)[0]
    # How many bumps are there?
    n_bumps.append(len(loc))


#### Add cut column and save

In [10]:
# Do any cuts overlap?
drop_index = [bad_filter_index , flat_jump_index , mismatch_index , red_index , variable_index , bad_i_index]
drop_name = ['bad_filter','flat_jump','mismatch','red','variable','bad_i']

# Re-Read in magnitude reduced files 
df = pd.read_csv(directory+'1_magnitude_reduced.csv') 

# Make a key
df['key'] = np.arange(df.shape[0])

# Add the number of bumps to the dataframe
df['n_bumps'] = n_bumps


# Make a cut column 
df['cut'] = ''

# Fill the cut column 
for index, name in zip(drop_index,drop_name):
    df.loc[df['key'].isin(index),'cut'] = name

print(f'{df[df.cut == ""].shape[0]} stars remaining')
print(f'{df[df.cut != ""].shape[0]} stars cut')
print(df[df.cut == ""].galaxy.value_counts())


# Save the dataframe
df.to_csv(directory+'2_sed_reduced.csv',index=False)
print('Saved')

4649 stars remaining
8704 stars cut
galaxy
lmc    2790
smc    1859
Name: count, dtype: int64
Saved


#### (Optional) Plot by cut

In [11]:
def plot(cutname,directory):
    pdf = df[df.cut == cutname].copy().reset_index(drop=True)
    n_plots = np.ceil(pdf.shape[0] / 20)
    sed_columns = ['uvw2_mag_dered','uvm2_mag_dered','uvw1_mag_dered','U_dered','B_dered','V_dered','I_dered']
    err_columns = ['uvw2_mag_err','uvm2_mag_err','uvw1_mag_err','e_U','e_B','e_V','e_I']
    #Approximate central wavelengths in angstroms
    eff_wav = [1928,2246,2600,3639.3,4350.6,5369.6,7609.2] 

    start = 0
    stop = 20
    for i in np.arange(n_plots):
        f, axes = plt.subplots(5, 4, figsize=(20, 20))
        for ax, n in zip(axes.flatten(),np.arange(start,stop)):
            if n >= pdf.shape[0]:
                break
            row = pdf.iloc[n]
            sed = row[sed_columns].values
            err = row[err_columns].values
            ax.plot(eff_wav,sed,ls='--',color='black',alpha=0.5)
            ax.errorbar(eff_wav,sed,yerr=err,fmt='o',color='red')
            ax.set_xticks(eff_wav)
            ax.set_xticklabels(['W2','M2','W1','U','B','V','I'])
            ax.set_ylim(20,14)
            ax.grid(alpha=0.4)
            ax.set_title(f'{row.key} - {row.cut}')

        plt.savefig(f'{directory}{int(i)}.png',bbox_inches='tight')
        plt.close()
        start += 20
        stop += 20

make_plots = False
if make_plots:
    for name in drop_name:
        print(name)
        plot(name,f'Plots/Cuts/{name}/')